In [75]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from collections import defaultdict
from sklearn.metrics.pairwise import euclidean_distances
import pickle

In [76]:
def subset_stock_data(data, start_date, end_date, verbose=False):
    """
    Subsets the given dataframe based on a specified date range.

    Args:
        data (pandas.DataFrame): The dataframe containing the stock data.
        start_date (str or pandas.Timestamp): The start date of the desired date range.
        end_date (str or pandas.Timestamp): The end date of the desired date range.
        verbose (bool, optional): If True, prints a success message. Defaults to False.

    Returns:
        pandas.DataFrame: The subset of the dataframe based on the specified date range.
        
    Raises:
        ValueError: If the dataframe does not contain a 'Date' column.

    """
    # Check if 'Date' column exists in the dataframe
    if 'date' not in data.columns:
        raise ValueError("DataFrame does not contain a 'Date' column.")
     
    # Convert date columns to datetime if they are not already datetime objects
    if not isinstance(data['date'], pd.DatetimeIndex):
        data['date'] = pd.to_datetime(data['date'])
    
    if not isinstance(start_date, pd.Timestamp):
        start_date = pd.to_datetime(start_date)

    if not isinstance(end_date, pd.Timestamp):
        end_date = pd.to_datetime(end_date)

    # Subset the dataframe based on date range
    subset = data[(data['date'] >= start_date) & (data['date'] <=end_date)]
    if verbose:
        print(f'Successfully subsetted data from {start_date} to {end_date}.')
    return subset

In [77]:
def handle_nans(df, interpolate=True):

    print("Mising value data before handling from 2008 onwards: ")

    # subset data from 2008 onwards
    df = df[df['date'].dt.year >= 2008]

    # set the print limit to 100
    pd.set_option('display.max_rows', 1000)

    # find percentage of missing values per column
    missing_values = (df.isnull().sum() / len(df)) * 100

    # print the columns with missing values in descending order
    print(f'\nPercentage of missing values per column:\n{missing_values.sort_values(ascending=False)[:15]}')

    # print the total number of missing values
    print('\nTotal number of missing values: ', df.isnull().sum().sum())

    # count the number of rows with missing values
    print('Number of rows with missing values: ', df.isnull().any(axis=1).sum())

    print("\n\n---------------------------------")
    print("---------------------------------")
    print("Handling missing values...")

    df_interpolated = df.copy()
    if interpolate:         
        
        # find unique permnos
        permnos = df_interpolated['permno'].unique()
        len_permnos = len(permnos)

        # interpolate missing values for each permno
        print('')
        for i, permno in enumerate(permnos):
            df_interpolated.loc[df_interpolated['permno'] == permno] = df_interpolated.loc[df_interpolated['permno'] == permno].interpolate(method='linear', limit_direction='forward', limit=5)
            print(f'Interpolating by permno ({i+1}/{len_permnos})...\r', flush=True, end='')

        print('')

        # print the number of missing values after interpolation
        print('Number of missing values after interpolation: ', df_interpolated.isnull().sum().sum(), 'instead of ', df.isnull().sum().sum())

        # print the number of rows with missing values after interpolation
        print('Number of rows with missing values after interpolation: ', df_interpolated.isnull().any(axis=1).sum(), 'instead of ', df.isnull().any(axis=1).sum())

        # find percentage of missing values per column after interpolation
        missing_values_interpolated = (df_interpolated.isnull().sum() / len(df_interpolated)) * 100

        # print the columns with missing values in descending order after interpolation
        print(f'\nPercentage of missing values per column after interpolation:\n{missing_values_interpolated.sort_values(ascending=False)[:15]}')

    return df_interpolated    
    

In [78]:
def remove_non_numerical_columns(data, verbose=False):
    """
    Remove non-numerical columns from a dataframe.

    Parameters:
    - data: pandas DataFrame
        The input dataframe from which non-numerical columns will be removed.
    - verbose: bool, optional
        If True, print a message with the deleted columns. Default is False.

    Returns:
    - pandas DataFrame
        The dataframe with non-numerical columns removed.
    """

    # Check first 10 rows for numerical columns
    first_10_rows = data.head(10)
    non_numerical_columns = []

    # Iterate through columns
    for column in data.columns:
        # Check if the column contains numerical data
        if pd.api.types.is_numeric_dtype(first_10_rows[column]):
            continue
        else:
            non_numerical_columns.append(column)

    # Remove non-numerical columns from the dataframe
    data = data.copy()
    data.drop(columns=non_numerical_columns, inplace=True)

    # drop date column if it exists
    if 'date' in data.columns:
        data.drop(columns=['date'], inplace=True)

    # Print message with deleted columns
    if verbose:
        if non_numerical_columns:
            print("Successfully removed columns with non-numerical values:", non_numerical_columns)

    return data

In [79]:
def preprocess_data(data, interpolate = False, start_date=None, end_date=None, verbose=False):
    """
    Preprocesses the input data by performing the following steps:
    1. Subset the data based on the specified start and end dates.
    2. Remove non-numerical columns from the subsetted data.
    3. Scale the numerical data using StandardScaler.

    Args:
        data (pd.DataFrame): The input data to be preprocessed.
        start_date (str, optional): The start date for subsetting the data. Defaults to None.
        end_date (str, optional): The end date for subsetting the data. Defaults to None.
        verbose (bool, optional): Whether to print verbose output. Defaults to False.

    Returns:
        np.ndarray: The preprocessed and scaled data.
    """
    if type(data) != pd.DataFrame:
        raise Exception('data must be a pandas dataframe')
    
    subset_data = subset_stock_data(data, start_date, end_date, verbose=verbose)
    subset_numerical_data = remove_non_numerical_columns(subset_data, verbose=verbose)

    if interpolate:
        subset_numerical_data = subset_numerical_data.interpolate(limit_direction='forward')

    if 'permno' in subset_numerical_data.columns:
        subset_numerical_data = subset_numerical_data.drop(columns=['permno'])

    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(subset_numerical_data)

    # make a DataFrame with the scaled data
    scaled_data = pd.DataFrame(scaled_data, columns=subset_numerical_data.columns)
        
    if verbose:
        print('Successfully scaled data.')
        
    return scaled_data

In [80]:
def find_q(explained_variance, required_explained_var = 0.95):
    """
    Finds the minimum number of principal components (q) required to explain a given amount of variance.

    Parameters:
    explained_variance (list): A list of explained variances for each principal component.
    required_explained_var (float): The required amount of variance to be explained (default is 0.95).

    Returns:
    int: The minimum number of principal components required to explain the given amount of variance.
    """
    cumulative_expl_var = np.cumsum(explained_variance)
    for i,j in enumerate(cumulative_expl_var):
        if j >= required_explained_var:
            q = i+1
            break 
    return q

In [81]:
def fit_pca(data):
    pca = PCA()
    pca.fit(data)
    return [pca.explained_variance_ratio_, pca.components_]

In [82]:
def fit_pfa(data, principal_components, q, diff_n_features):
    """
    Perform feature selection using Principal Feature Analysis (PFA).

    Parameters:
    - data: numpy array
        The input data matrix.
    - principal_components: numpy array
        The principal components obtained from PCA.
    - q: int
        The number of principal components to consider.
    - diff_n_features: int
        The difference between the number of features to select and the number of principal components.

    Returns:
    - indices: list
        The indices of the selected features.
    - features: numpy array
        The selected features from the input data matrix.
    """
    A_q = principal_components.T[:,:q]
    clusternumber = min([q + diff_n_features, data.shape[1]])
        
    kmeans = KMeans(n_clusters = clusternumber).fit(A_q)
    clusters = kmeans.predict(A_q)
    cluster_centers = kmeans.cluster_centers_

    dists = defaultdict(list)
    for i, c in enumerate(clusters):
        dist = euclidean_distances([A_q[i, :]], [cluster_centers[c, :]])[0][0]
        dists[c].append((i, dist))

    indices = [sorted(f, key=lambda x: x[1])[0][0] for f in dists.values()]
    features = data[:, indices]
    return indices, features

In [83]:
def transform_pca(data, fitted, principal_components, q, preprocess_data=None):
    """
    Transforms the input data using Principal Component Analysis (PCA).

    Args:
        data (array-like): The input data to be transformed.
        fitted (bool): Indicates whether the PCA model has been fitted to the data.
        principal_components (array-like): The principal components obtained from the PCA model.
        q (int): The number of principal components to keep in the transformed data.
        preprocess_data (function, optional): A function to preprocess the data before transformation.

    Returns:
        array-like: The transformed data with reduced dimensions.

    Raises:
        Exception: If the model has not been fitted to the data.
    """
    if preprocess_data is not None:
        scaled_data = preprocess_data(data)
    else:
        scaled_data = data

    if not fitted:
        raise Exception('The model has not been fitted to the data.')

    reduced_data = np.matmul(np.array(scaled_data), np.transpose(principal_components))[:, :q]
    return reduced_data

def transform_pfa(data, fitted, features, preprocess_data=None):
    if preprocess_data != None:
        scaled_data = preprocess_data(data)
    else:
        scaled_data = data

    if fitted != True:
        raise Exception('The model has not been fitted to the data.')
    return features

In [84]:
def fit_transform(data, method):
    """
    Applies feature selection to the input data using the specified method.

    Args:
        data (numpy.ndarray): The input data to be transformed.
        method (str): The feature selection method to be used. Must be either 'pca' or 'pfa'.

    Returns:
        numpy.ndarray: The transformed data after applying feature selection.

    Raises:
        Exception: If the method is not 'pca' or 'pfa'.
    """
    if method not in ['pca', 'pfa']:
        raise Exception("Method must be either 'pca' or 'pfa'")
    scaled_data = preprocess_data(data)
    if method == 'PCA':
        explained_variance, principal_components = fit_pca(scaled_data)
        q = find_q(explained_variance)
        output = transform_pca(scaled_data, True, principal_components, q)
    elif method == 'PFA':
        explained_variance, principal_components = fit_pca(scaled_data)
        q = find_q(explained_variance)
        diff_n_features = 0
        indices, features = fit_pfa(scaled_data, principal_components, q, diff_n_features)
        output = transform_pfa(scaled_data, True, features)
    return output

In [85]:
# open data
stationary_data = pd.read_csv('../../data/datasetlabel.csv')

# set the 'date' column as datetime 
stationary_data['date'] = pd.to_datetime(stationary_data['date'])

In [86]:
# subset stationary data from 2008 onwards and intepolate missing values
stationary_data = handle_nans(stationary_data)

Mising value data before handling from 2008 onwards: 

Percentage of missing values per column:
short_debt      4.044770
sale_equity     3.784711
dltt_be         3.705148
roe             3.703360
bm              3.678882
ptb             3.678627
pay_turn        3.117387
fcf_ocf         2.698840
rect_turn       1.240356
pe_exi          1.168967
pe_op_dil       1.167817
pe_inc          1.160751
pe_op_basic     1.156238
lt_ppent        1.004775
debt_capital    0.814914
dtype: float64

Total number of missing values:  1026036
Number of rows with missing values:  391624


---------------------------------
---------------------------------
Handling missing values...

Interpolating by permno (715/715)...
Number of missing values after interpolation:  1013695 instead of  1026036
Number of rows with missing values after interpolation:  388691 instead of  391624

Percentage of missing values per column after interpolation:
short_debt      4.023698
sale_equity     3.754614
dltt_be         3.66440

In [87]:
nan_threshold = 0.03705147

# find the number of nan values in each column
nan_values = stationary_data.isna().sum()

# find the columns with more nan values than the threshold
columns_to_drop = list(nan_values[nan_values > (nan_threshold * stationary_data.shape[0])].index) + ['target'] # can't include tartet in the PFA
key_features = ['MACD_index', 'vol', 'ret', 'prc_adj', 'retx', 'ret_industry_relative'] #, 'interest_rates']

# drop the columns with more nan values than the threshold
print(f'dropping {len(columns_to_drop)} columns due to containing too many NaNs or being the target:\n', (columns_to_drop))
print(f'dropping {len(key_features)} key features:\n', (key_features))
stationary_data = stationary_data.drop(columns=(columns_to_drop + key_features))

dropping 3 columns due to containing too many NaNs or being the target:
 ['short_debt', 'sale_equity', 'target']
dropping 6 key features:
 ['MACD_index', 'vol', 'ret', 'prc_adj', 'retx', 'ret_industry_relative']


In [88]:
# set the print limit to 100
pd.set_option('display.max_rows', 1000)

# find percentage of missing values per column
missing_values = (stationary_data.isnull().sum() / len(stationary_data)) * 100

# print the columns with missing values in descending order
print(f'Percentage of missing values per column:\n{missing_values.sort_values(ascending=False)}')


print('Total number of missing values: ', stationary_data.isnull().sum().sum())

# count the number of rows with missing values
print('Number of rows with missing values: ', stationary_data.isnull().any(axis=1).sum())

Percentage of missing values per column:
dltt_be             3.664409
bm                  3.653255
ptb                 3.653000
roe                 3.650361
pay_turn            3.103339
fcf_ocf             2.653418
rect_turn           1.231417
pe_exi              1.150321
pe_op_dil           1.149172
pe_inc              1.142957
pe_op_basic         1.138657
lt_ppent            0.996048
debt_capital        0.811295
aftret_invcapx      0.756465
debt_ebitda         0.687800
cash_debt           0.602661
roce                0.486956
totdebt_invcap      0.403519
CAPEI               0.399901
evm                 0.382277
debt_invcap         0.357799
debt_at             0.344986
capital_ratio       0.299393
lt_debt             0.299308
cfm                 0.236518
aftret_equity       0.136692
aftret_eq           0.136606
roa                 0.136436
at_turn             0.136351
accrual             0.127795
pcf                 0.127752
sale_invcap         0.110979
de_ratio            0.107914
de

In [89]:
# drop rows with missing values
stationary_data_full = stationary_data.dropna()

In [90]:
# find the number of rows per year
rows_per_year = stationary_data_full['date'].dt.year.value_counts().sort_index()

# print the number of rows per year
print('Number of rows per year:\n', rows_per_year)

Number of rows per year:
 date
2008    131504
2009    133722
2010    134305
2011    135624
2012    134713
2013    135198
2014    135953
2015    133633
2016    128825
2017    125501
2018    123914
2019    121660
2020    120178
2021    117101
2022    115893
2023    113016
Name: count, dtype: int64


In [91]:
def find_features(data, start_date, period_duration, periods, explained_variance_threshold=0.95, diff_n_features=2, key_features=None):
    start_date = pd.to_datetime(start_date)

    assert data.isnull().sum().sum() == 0, 'Data contains missing values.'

    # create a dictionary to store the features
    features_dict = {}
    
    for i in range(periods):
        new_start_date = start_date + pd.DateOffset(years=(period_duration*i))
        end_date = new_start_date + pd.DateOffset(years=period_duration) - pd.DateOffset(days=1)

        # preprocess the data
        scaled_data = preprocess_data(data.copy(), start_date=new_start_date, end_date=end_date, verbose=True)

        # fit the pca model
        explained_variance, principal_components = fit_pca(scaled_data)

        # find the number of principal components to explain the variance threshold
        q = find_q(explained_variance, explained_variance_threshold)
        print(f'Number of principal components to explain {explained_variance_threshold*100}% of the variance: {q}')

        # fit the pfa model
        indices, features = fit_pfa(np.array(scaled_data), principal_components, q, diff_n_features)
        
        # find the list of features
        features_list = list(scaled_data.columns[indices])
        if key_features != None:
            features_list = features_list + key_features

        # store the features in the dictionary
        features_dict[new_start_date] = features_list

        print(f'Succesfully extracted features for period starting in {new_start_date}.\n')

    return features_dict

In [92]:
start_date = '2008-01-01' # start_date of the first period
period_duration = 2 # duration of each period in years
periods =  6 # number of periods
explained_variance_threshold = 0.8 # threshold for explained variance
diff_n_features = 2 # difference between the number of features to select and the number of principal components

features = find_features(stationary_data_full, start_date, period_duration, periods, explained_variance_threshold, diff_n_features, key_features)

# save the features as a dictionary with pickle
with open('../../data/selected_features.pkl', 'wb') as f:
    pickle.dump(features, f)

Successfully subsetted data from 2008-01-01 00:00:00 to 2009-12-31 00:00:00.
Successfully removed columns with non-numerical values: ['date']
Successfully scaled data.
Number of principal components to explain 80.0% of the variance: 22
Succesfully extracted features for period starting in 2008-01-01 00:00:00.

Successfully subsetted data from 2010-01-01 00:00:00 to 2011-12-31 00:00:00.
Successfully removed columns with non-numerical values: ['date']
Successfully scaled data.
Number of principal components to explain 80.0% of the variance: 22
Succesfully extracted features for period starting in 2010-01-01 00:00:00.

Successfully subsetted data from 2012-01-01 00:00:00 to 2013-12-31 00:00:00.
Successfully removed columns with non-numerical values: ['date']
Successfully scaled data.
Number of principal components to explain 80.0% of the variance: 22
Succesfully extracted features for period starting in 2012-01-01 00:00:00.

Successfully subsetted data from 2014-01-01 00:00:00 to 2015-12-

In [93]:
# load the features from the pickle file
with open('../../data/selected_features.pkl', 'rb') as f:
    features_dict = pickle.load(f)

for key, value in features_dict.items():
    print(f'Selected features for period starting in: {key}:')
    print(value)
    print('')

Selected features for period starting in: 2008-01-01 00:00:00:
['CAPEI', 'debt_assets', 'debt_ebitda', 'pe_op_basic', 'pe_inc', 'ps', 'pcf', 'ptpm', 'roce', 'aftret_invcapx', 'aftret_equity', 'GProf', 'capital_ratio', 'cash_lt', 'fcf_ocf', 'de_ratio', 'sale_invcap', 'rect_turn', 'pay_turn', 'adv_sale', 'naics_processed', 'stat_divyeld', 'mktcap', 'ret_industry_tot', 'MACD_index', 'vol', 'ret', 'prc_adj', 'retx', 'ret_industry_relative']

Selected features for period starting in: 2010-01-01 00:00:00:
['CAPEI', 'debt_assets', 'evm', 'pe_op_basic', 'pe_exi', 'ps', 'pcf', 'npm', 'opmad', 'roce', 'aftret_eq', 'debt_at', 'cash_lt', 'fcf_ocf', 'staff_sale', 'de_ratio', 'at_turn', 'rect_turn', 'pay_turn', 'adv_sale', 'stat_divyeld', 'prc', 'mktcap', 'ret_industry_tot', 'MACD_index', 'vol', 'ret', 'prc_adj', 'retx', 'ret_industry_relative']

Selected features for period starting in: 2012-01-01 00:00:00:
['CAPEI', 'opmbd', 'debt_ebitda', 'pe_op_dil', 'pe_inc', 'ps', 'npm', 'roce', 'roe', 'aftret

In [94]:
# find features that are in all periods
features_in_all_periods = []

for key, value in features_dict.items():
    if len(features_in_all_periods) == 0:
        features_in_all_periods = value
    else:
        features_in_all_periods = [f for f in features_in_all_periods if f in value]

print(f'There are {len(features_in_all_periods)} features that are selected in all periods.')
print('Features that are selected in all periods but not key feaures:')
print(list(set(features_in_all_periods) - set(key_features)))
print('Key features:')
print(key_features)

There are 12 features that are selected in all periods.
Features that are selected in all periods but not key feaures:
['fcf_ocf', 'de_ratio', 'CAPEI', 'cash_lt', 'ps', 'rect_turn']
Key features:
['MACD_index', 'vol', 'ret', 'prc_adj', 'retx', 'ret_industry_relative']


In [95]:
# find how many times each feature is selected
feature_counts = {}
for key, value in features_dict.items():
    for feature in value:
        if feature in feature_counts:
            feature_counts[feature] += 1
        else:
            feature_counts[feature] = 1

# order the counts by value in descending order
feature_counts = dict(sorted(feature_counts.items(), key=lambda item: item[1], reverse=True))

# print the feature counts
print('Feature counts:')
for key, value in feature_counts.items():
    print(f'{key}: {value}')

Feature counts:
CAPEI: 6
ps: 6
cash_lt: 6
fcf_ocf: 6
de_ratio: 6
rect_turn: 6
MACD_index: 6
vol: 6
ret: 6
prc_adj: 6
retx: 6
ret_industry_relative: 6
pay_turn: 5
adv_sale: 5
mktcap: 5
staff_sale: 5
roce: 4
at_turn: 4
pe_op_dil: 4
debt_ebitda: 3
pe_inc: 3
pcf: 3
ptpm: 3
aftret_invcapx: 3
aftret_equity: 3
stat_divyeld: 3
ret_industry_tot: 3
evm: 3
pe_exi: 3
aftret_eq: 3
debt_at: 3
prc: 3
debt_capital: 3
divyield: 3
rsi: 3
debt_assets: 2
pe_op_basic: 2
GProf: 2
capital_ratio: 2
sale_invcap: 2
npm: 2
opmad: 2
opmbd: 2
roe: 2
roa: 2
dltt_be: 2
naics_processed: 1
equity_invcap: 1
lt_debt: 1
rd_sale: 1
bm: 1
